In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python3 -c "import monai" || pip install -q "monai"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import random
import shutil
import tempfile
from monai.config import print_config
from monai.apps import download_and_extract

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmptwamkqlu


In [ ]:
msd_task = "Task09_Spleen"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"

compressed_file = os.path.join(root_dir, msd_task + ".tar")
dataroot = os.path.join(root_dir, msd_task)

if not os.path.exists(dataroot):
    download_and_extract(resource, compressed_file, root_dir)

Task09_Spleen.tar: 1.50GB [01:32, 17.3MB/s]                            

2024-07-17 21:44:25,048 - INFO - Downloaded: /tmp/tmptwamkqlu/Task09_Spleen.tar
2024-07-17 21:44:25,049 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmptwamkqlu/Task09_Spleen.tar.
2024-07-17 21:44:25,050 - INFO - Writing into directory: /tmp/tmptwamkqlu.


In [1]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

Cloning into 'nnUNet'...
remote: Enumerating objects: 12735, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 12735 (delta 8), reused 12 (delta 4), pack-reused 12717
Receiving objects: 100% (12735/12735), 7.71 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (9769/9769), done.


In [ ]:
!cd nnUNet && pip install -e .

Obtaining file:///content/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 42.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import glob as gb
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib
import time
import gc
import os
import copy
from collections import defaultdict
from scipy.ndimage import distance_transform_edt as edt
import shutil
import torch.nn.functional as F
from nnUNet.nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
import SimpleITK as sitk
from scipy import ndimage
import nibabel as nib
import pydicom
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from ipywidgets import interact
from ipywidgets.widgets import IntSlider
import cv2

id = 99

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):
        if overwrite:
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)
    else:
        os.makedirs(folder_path)

working_dir = "."
nnunet_dir = "nn_UNet/nnUNet_raw"
nnunet_prepocess = "nn_UNet/nnUNet_preprocessed"
nnunet_result = "nn_UNet/nnUNet_results"
task_name = f'Dataset0{id}_CHD'
main_dir = os.path.join(working_dir, nnunet_dir)
task_folder_name = os.path.join(main_dir, task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
for i in [main_dir, task_folder_name, train_image_dir, train_label_dir]:
    print(i)
    make_if_dont_exist(i)
make_if_dont_exist(nnunet_prepocess)
make_if_dont_exist(nnunet_result)
os.environ['nnUNet_raw'] = os.path.join(working_dir, nnunet_dir)
os.environ['nnUNet_preprocessed'] = os.path.join(working_dir, nnunet_prepocess)
os.environ['nnUNet_results'] = os.path.join(working_dir,nnunet_result)

./nn_UNet/nnUNet_raw
./nn_UNet/nnUNet_raw/Dataset099_CHD
./nn_UNet/nnUNet_raw/Dataset099_CHD/imagesTr
./nn_UNet/nnUNet_raw/Dataset099_CHD/labelsTr


In [ ]:
test_dir = os.path.join(dataroot, "imagesTs/")
train_dir = os.path.join(dataroot, "imagesTr/")
label_dir = os.path.join(dataroot, "labelsTr/")
dataset = os.path.join(dataroot, "dataset.json")

In [ ]:
def copy_and_format(in_file: str, out_file: str):
    img = sitk.ReadImage(in_file)
    sitk.WriteImage(img, out_file)
remove = len(".nii.gz")
for file in tqdm(os.listdir(train_dir)):
    flie_path = os.path.join(train_dir, file)
    if file.endswith(".nii.gz") and not file.startswith("._"):
        dest = copy_and_format(flie_path, os.path.join(train_image_dir, file[:-remove] + "_0000.nii"))

for file in tqdm(os.listdir(label_dir)):
    flie_path = os.path.join(label_dir, file)
    if file.endswith(".nii.gz") and not file.startswith("._"):
        dest = copy_and_format(flie_path, os.path.join(train_label_dir, file[:-remove] + ".nii"))
num_train = sum([len(x[2]) for x in list(os.walk(train_image_dir))])
print(num_train)
print(sum([len(x[2]) for x in list(os.walk(train_label_dir))]))

100%|██████████| 82/82 [00:01<00:00, 46.49it/s]

41
41


In [ ]:
labels = {
        "background": 0,
        "spleen": 1
    }
generate_dataset_json(task_folder_name, {0: 'RGB'}, labels, num_train, '.nii', overwrite_image_reader_writer='NibabelIO', dataset_name=task_name)

In [ ]:
!nnUNetv2_plan_and_preprocess -d 0{id} --verify_dataset_integrity  -pl nnUNetPlannerResEncL

Fingerprint extraction...
Dataset099_CHD
Using <class 'nnunetv2.imageio.nibabel_reader_writer.NibabelIO'> reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [1.60001004 0.81675806 0.81675806]. 
Current patch size: (112, 256, 256). 
Current median shape: [187.         497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [1.64801034 0.8412608  0.8412608 ]. 
Current patch size: (112, 256, 256). 
Current median shape: [181.55339806 482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [1.69745065 0.86649862 0.86649862]. 
Current patch size: (112, 256, 256). 
Current median shape: [176.26543501 468.55252959 468.55252959]
Attempting to find 3d_lowres config. 
Current spacing: [1.74837417 0.89249358 0.89249358]. 
Current patch size: (112, 256, 256).

In [ ]:
# folder = os.path.join(working_dir, nnunet_prepocess, task_name, "nnUNetPlans_3d_fullres")
# for file in os.listdir(folder):
#     file_path = os.path.join(folder, file)
#     if file_path.endswith(".npy"):
#         os.remove(file_path)

In [ ]:
!nnUNetv2_train 0{id} 3d_fullres 0 -p nnUNetResEncUNetLPlans

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-07-17 23:20:50.615744: do_dummy_2d_data_aug: True
2024-07-17 23:20:50.616250: Using splits from existing split file: ./nn_UNet/nnUNet_preprocessed/Dataset099_CHD/splits_final.json
2024-07-17 23:20:50.616452: The split file contains 5 splits.
2024-07-17 23:20:50.616509: Desired fold for training: 0
2024-07-17 23:20:50.616555: This split has 32 training and 9 validation cases.
using pin_memory on device 0
using pin_memory on device 0
2024-07-17 23:21:14.082734: Using torch.compile...
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: